# ETL - Productos - **CHardy Tecno Store**
---
## 1. Extracción.
### 1.1. importamos el DataFrame productos.csv

In [1]:
import pandas as pd
from pathlib import Path

# Ruta al archivo Parquet
ruta_archivo = Path("../data/raw/productos.csv")

try:
    if not ruta_archivo.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo.resolve()}")
    
    df = pd.read_csv(ruta_archivo)

    print(" Extracción correcta")
    print(f" Registros cargados: {len(df)}")
    print(f" Columnas: {len(df.columns)}")


except FileNotFoundError as e:
    print(f"❌ Error: {e}")
except Exception as e:
    print(f"⚠️ Error inesperado: {e}")

 Extracción correcta
 Registros cargados: 28
 Columnas: 5


## 2. Transformación
### 2.1. Verificamos los Datos Extraidos

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   producto_id           28 non-null     int64 
 1   nombre_producto       28 non-null     object
 2   categoria             28 non-null     object
 3   subcategoria          28 non-null     object
 4   precio_base_ars_2018  28 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 1.2+ KB


In [3]:
df.head()

,producto_id,nombre_producto,categoria,subcategoria,precio_base_ars_2018
0,1,Laptop Premium,Tecnologia,Computo,30000
1,2,Desktop Gamer,Tecnologia,Computo,40000
2,3,"Monitor 27""",Tecnologia,Computo,8000
3,4,Teclado Mecánico,Tecnologia,Computo,2500
4,5,Mouse Gamer,Tecnologia,Computo,1500


In [4]:
# downcast="float" intenta usar float32 en lugar de float64 para ahorrar memoria.
df["precio_base_ars_2018"] = pd.to_numeric(df["precio_base_ars_2018"], downcast="float")
df.dtypes

producto_id               int64
nombre_producto          object
categoria                object
subcategoria             object
precio_base_ars_2018    float32
dtype: object

In [5]:
# Conversión defensiva de columnas object
for col in ["nombre_producto", "categoria", "subcategoria"]:
    df[col] = df[col].astype("string")  # o "category" si son pocos valores

print("Tipos después de conversión:\n", df.dtypes)

Tipos después de conversión:
 producto_id                      int64
nombre_producto         string[python]
categoria               string[python]
subcategoria            string[python]
precio_base_ars_2018           float32
dtype: object


In [6]:
# Filtrar duplicados (todas las columnas)
df_duplicados = df[df.duplicated()]
print(df_duplicados)

df_duplicados_id = df[df.duplicated(subset=["producto_id"], keep=False)]
print("producto_id (duplicados):\n", df_duplicados_id)


Empty DataFrame
Columns: [producto_id, nombre_producto, categoria, subcategoria, precio_base_ars_2018]
Index: []
producto_id (duplicados):
 Empty DataFrame
Columns: [producto_id, nombre_producto, categoria, subcategoria, precio_base_ars_2018]
Index: []


In [7]:
print(df.isnull().sum()) # Muestra valores nulos (valor cero no es nulo)

producto_id             0
nombre_producto         0
categoria               0
subcategoria            0
precio_base_ars_2018    0
dtype: int64


In [8]:
# Valores únicos de la columna 'categaria'
valores_unicos = df["categoria"].unique()
valores_unicos_sub = df["subcategoria"].unique()
print("Categorías:\n", valores_unicos)
print("SubCategorías:\n", valores_unicos_sub)

Categorías:
 <StringArray>
['Tecnologia', 'Linea Blanca', 'Pequeños Electro']
Length: 3, dtype: string
SubCategorías:
 <StringArray>
[             'Computo',              'Moviles', 'TV & Entretenimiento',
        'Refrigeracion',               'Lavado',               'Cocina',
        'Climatizacion',       'Cocina & Hogar',     'Cuidado Personal']
Length: 9, dtype: string


In [ ]:
# Conversión defensiva de columnas object
for col in ["nombre_producto", "categoria", "subcategoria"]:
    df[col] = df[col].astype("string")  # o "category" si son pocos valores

print("Tipos después de conversión:\n", df.dtypes)

## 3. Carga.
### 3.1. Guardamos la dimensión en formato parquet.

In [9]:
import os

filename = "../data/processed/dim_productos.parquet"

try:
    df.to_parquet(filename, engine="fastparquet")
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")

✔️ Dataset guardado con éxito en: ../data/processed/dim_productos.parquet
